In [8]:
import re
import pandas as pd
import numpy as np
import subprocess
import scipy.stats as st
from scipy.optimize import minimize
import scipy.stats as stats
from scipy.interpolate import make_interp_spline
import seaborn
import os
from functools import reduce
import itertools
import math

from liftover import get_lifter

In [9]:
import matplotlib as mpl 
mpl.use('Agg')
mpl.rcParams['pdf.fonttype'] = 42
import seaborn as sns
import matplotlib.pyplot as plt
plt.close("all")
mpl.rcParams['font.sans-serif'] = "Arial"
mpl.rcParams["font.family"] = 'sans-serif'
mpl.rcParams['font.size'] = 8
sns.set(style="ticks")

In [10]:
cd /Users/davidy/jamboree20crispr/tad_analysis

/Users/davidy/jamboree20crispr/tad_analysis


In [11]:
### Prepare K562 element quantification file
k562_eq = pd.read_csv('k562.elementQuant.combined.hg38.addpValueEffectSize.20230614.txt', sep = '\t')

# Subset to just elements with reported significance
k562_eq = k562_eq[~k562_eq['pValueAdjusted'].isna()]
display(k562_eq)

# Write to output 
k562_eq.to_csv('k562_eq_combined_hg38.bed', sep = '\t', header = False, index = False)

### Prepare TAD file (hg38)
k562_tad_hg38 = pd.read_csv('ENCFF173VDJ.bedpe', sep = '\t', skiprows=[1])

# There are two sets of chromcoords that look similar on a spotcheck. Formally check if any are NOT equal
# This below checks out. No need to run it again.
#notequal_k562_tad_hg38 = k562_tad_hg38[(k562_tad_hg38[['x1','y1']].nunique(axis=1) == 1)&(k562_tad_hg38[['x2','y2']].nunique(axis=1) == 1)]
# Because the two chromcoords are equal, I can just keep the first 3 cols for bedtools purposes
k562_tad_hg38_bed = k562_tad_hg38[['#chr1','x1','x2']]
k562_tad_hg38_bed['tad_cc'] = k562_tad_hg38_bed['#chr1'].astype(str) + '_' + k562_tad_hg38_bed['x1'].astype(str) + '_' + k562_tad_hg38_bed['x2'].astype(str)


# Write to output
k562_tad_hg38_bed.to_csv('k562_tad_hg38.bed', sep = '\t', header = False, index=False)






,chr,start,end,label,EffectSize,pValue,pValueAdjusted
113,chrX,48521437,48521957,chrX:48379825-48380345:._EBP,0.321735,4.510889e-43,7.803839e-41
114,chrX,48910928,48911588,chrX:48768205-48768865:._OTUD5,-0.140960,1.023842e-03,4.530503e-02
115,chrX,48940454,48940954,chrX:48797725-48798225:._OTUD5,0.298665,1.120495e-04,9.916381e-03
116,chrX,48957383,48958744,chrX:48814645-48816005:._OTUD5,0.170512,8.116472e-04,4.530503e-02
117,chrX,49862042,49862582,chrX:49626645-49627185:._OTUD5,-0.358200,8.059167e-08,1.426473e-05
...,...,...,...,...,...,...,...
317,chr19,48746728,48747508,chr19:49249985-49250765:._FUT1,0.164291,1.103408e-49,4.303290e-48
318,chr19,48747828,48748368,chr19:49251085-49251625:._FUT1,0.331336,1.013670e-89,5.271085e-88
319,chr19,48753488,48754588,chr19:49256745-49257845:._FUT1,0.472751,2.551523e-104,1.990188e-102
320,chr19,48755248,48755768,chr19:49258505-49259025:._FUT1,0.336078,9.140233e-126,1.425876e-123


<ipython-input-11-e7e31ebd4e9e>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  k562_tad_hg38_bed['tad_cc'] = k562_tad_hg38_bed['#chr1'].astype(str) + '_' + k562_tad_hg38_bed['x1'].astype(str) + '_' + k562_tad_hg38_bed['x2'].astype(str)


In [12]:
### Determine if any elements are NOT in a TAD at all. Exclude these from analysis
# First find the ones that dont intersect. Just to keep on record
eq_notintersect_file = 'k562_alleq_notintad.bed'
fopen = open(eq_notintersect_file, 'w')
subprocess.call(['bedtools','intersect', '-a', 'k562_eq_combined_hg38.bed', '-b', 'k562_tad_hg38.bed', '-v'], stdout = fopen)
fopen.close()

# Check if file is empty
if os.stat(eq_notintersect_file).st_size == 0:
    print('There are no significant elements that are not in a TAD')

else:
    print('The following element quantification elements lack intersections with TADs')
    eq_notintad_df = pd.read_csv(eq_notintersect_file, sep = '\t', header = None, names = ['chr','start','end','label','EffectSize','pValue','pValueAdjusted'])
    display(eq_notintad_df)

### Get the ones that actually intersect
eq_intersect_file = 'k562_alleq_intad.bed'
fopen = open(eq_intersect_file, 'w')
subprocess.call(['bedtools','intersect', '-a', 'k562_eq_combined_hg38.bed', '-b', 'k562_tad_hg38.bed', '-wa','-wb'], stdout = fopen)
fopen.close()

eq_intad_df = pd.read_csv(eq_intersect_file, sep = '\t', header = None, names = ['chr','start','end','label','EffectSize','pValue','pValueAdjusted','chr2','start2','end2', 'tad_cc'])
display(eq_intad_df)

## Create a new column for gene name
eq_intad_df['symbol'] = eq_intad_df['label'].str.split('_').str[-1]
display(eq_intad_df)

all_symbols = set(eq_intad_df['symbol'].tolist())
print(all_symbols)
print(len(all_symbols))


The following element quantification elements lack intersections with TADs


,chr,start,end,label,EffectSize,pValue,pValueAdjusted
0,chrX,48521437,48521957,chrX:48379825-48380345:._EBP,0.321735,4.510889e-43,7.803839e-41
1,chrX,48910928,48911588,chrX:48768205-48768865:._OTUD5,-0.140960,1.023842e-03,4.530503e-02
2,chrX,48940454,48940954,chrX:48797725-48798225:._OTUD5,0.298665,1.120495e-04,9.916381e-03
3,chrX,48957383,48958744,chrX:48814645-48816005:._OTUD5,0.170512,8.116472e-04,4.530503e-02
4,chrX,49862042,49862582,chrX:49626645-49627185:._OTUD5,-0.358200,8.059167e-08,1.426473e-05
...,...,...,...,...,...,...,...
60,chrX,48784617,48785277,chrX:48643025-48643685:._HDAC6,-0.039307,6.917334e-04,2.559414e-02
61,chrX,48785717,48786837,chrX:48644125-48645245:._HDAC6,-0.306471,3.896652e-208,8.650568e-206
62,chrX,48800418,48800918,chrX:48658825-48659325:._HDAC6,0.136011,1.190781e-16,6.608836e-15
63,chrX,48801138,48802158,chrX:48659545-48660565:._HDAC6,0.168007,7.026803e-78,5.199834e-76


,chr,start,end,label,EffectSize,pValue,pValueAdjusted,chr2,start2,end2,tad_cc
0,chr19,12886091,12887931,chr19:12996905-12998745:._CALR,0.085682,2.343624e-13,1.335866e-11,chr19,12875000,12955000,chr19_12875000_12955000
1,chr19,12936451,12936951,chr19:13047265-13047765:._CALR,0.084431,7.167944e-04,2.042864e-02,chr19,12875000,12955000,chr19_12875000_12955000
2,chr19,12938211,12938711,chr19:13049025-13049525:._CALR,0.249025,1.401191e-33,1.597357e-31,chr19,12875000,12955000,chr19_12875000_12955000
3,chr19,12775491,12776011,chr19:12886305-12886825:._DNASE2,0.046131,4.548657e-04,1.758814e-02,chr19,12695000,12855000,chr19_12695000_12855000
4,chr19,12881331,12881831,chr19:12992145-12992645:._DNASE2,0.393293,7.151473e-153,8.295708e-151,chr19,12875000,12955000,chr19_12875000_12955000
...,...,...,...,...,...,...,...,...,...,...,...
214,chr19,48753488,48754588,chr19:49256745-49257845:._FUT1,0.472751,2.551523e-104,1.990188e-102,chr19,48630000,48990000,chr19_48630000_48990000
215,chr19,48753488,48754588,chr19:49256745-49257845:._FUT1,0.472751,2.551523e-104,1.990188e-102,chr19,48640000,48780000,chr19_48640000_48780000
216,chr19,48755248,48755768,chr19:49258505-49259025:._FUT1,0.336078,9.140233e-126,1.425876e-123,chr19,48630000,48990000,chr19_48630000_48990000
217,chr19,48755248,48755768,chr19:49258505-49259025:._FUT1,0.336078,9.140233e-126,1.425876e-123,chr19,48640000,48780000,chr19_48640000_48780000


,chr,start,end,label,EffectSize,pValue,pValueAdjusted,chr2,start2,end2,tad_cc,symbol
0,chr19,12886091,12887931,chr19:12996905-12998745:._CALR,0.085682,2.343624e-13,1.335866e-11,chr19,12875000,12955000,chr19_12875000_12955000,CALR
1,chr19,12936451,12936951,chr19:13047265-13047765:._CALR,0.084431,7.167944e-04,2.042864e-02,chr19,12875000,12955000,chr19_12875000_12955000,CALR
2,chr19,12938211,12938711,chr19:13049025-13049525:._CALR,0.249025,1.401191e-33,1.597357e-31,chr19,12875000,12955000,chr19_12875000_12955000,CALR
3,chr19,12775491,12776011,chr19:12886305-12886825:._DNASE2,0.046131,4.548657e-04,1.758814e-02,chr19,12695000,12855000,chr19_12695000_12855000,DNASE2
4,chr19,12881331,12881831,chr19:12992145-12992645:._DNASE2,0.393293,7.151473e-153,8.295708e-151,chr19,12875000,12955000,chr19_12875000_12955000,DNASE2
...,...,...,...,...,...,...,...,...,...,...,...,...
214,chr19,48753488,48754588,chr19:49256745-49257845:._FUT1,0.472751,2.551523e-104,1.990188e-102,chr19,48630000,48990000,chr19_48630000_48990000,FUT1
215,chr19,48753488,48754588,chr19:49256745-49257845:._FUT1,0.472751,2.551523e-104,1.990188e-102,chr19,48640000,48780000,chr19_48640000_48780000,FUT1
216,chr19,48755248,48755768,chr19:49258505-49259025:._FUT1,0.336078,9.140233e-126,1.425876e-123,chr19,48630000,48990000,chr19_48630000_48990000,FUT1
217,chr19,48755248,48755768,chr19:49258505-49259025:._FUT1,0.336078,9.140233e-126,1.425876e-123,chr19,48640000,48780000,chr19_48640000_48780000,FUT1


{'DHPS', 'FTL', 'PRDX2', 'RPN1', 'HNRNPA1', 'RAD23A', 'HDAC6', 'PPP1R15A', 'PQBP1', 'H1FX', 'CNBP', 'DNASE2', 'COPZ1', 'RNASEH2A', 'BAX', 'C19orf43', 'CALR', 'NFE2', 'ITGA5', 'RAB7A', 'PIM2', 'WDR83OS', 'GATA1', 'PLP2', 'FUT1', 'JUNB', 'BCAT2', 'KLF1', 'NUCB1'}
29


In [13]:
##### Get the TAD cc for the symbols
# Using Human gene refseq consensus coordinates
refseq_hg38 = pd.read_csv('/Users/davidy/misc_resources/1kb_promoter_human_gene_refseq_consensus_coordinates_engreitz_hg38.bed', sep = '\t', header = None, names = ['gchr','gstart','gend','symbol','strand'])
#display(refseq_hg38)

symbols_in_refseq = refseq_hg38[refseq_hg38['symbol'].isin(all_symbols)]
#display(symbols_in_refseq)
#print(len(symbols_in_refseq))

# Merge this dataframe with eq_intad_df
merged_eq_symbol_intad = eq_intad_df.merge(symbols_in_refseq, on='symbol')
#display(merged_eq_symbol_intad)

### Determine if symbol in same TAD
sametad_merged_eq_symbol_intad = merged_eq_symbol_intad.loc[(merged_eq_symbol_intad['chr2']==merged_eq_symbol_intad['gchr'])&(
    (merged_eq_symbol_intad['gstart']<=merged_eq_symbol_intad['end2'])&(merged_eq_symbol_intad['gend']>=merged_eq_symbol_intad['start2']))]
display(sametad_merged_eq_symbol_intad)

differenttad_merged_eq_symbol_intad = merged_eq_symbol_intad.loc[~(
    (merged_eq_symbol_intad['gstart']<=merged_eq_symbol_intad['end2'])&(merged_eq_symbol_intad['gend']>=merged_eq_symbol_intad['start2']))]
display(differenttad_merged_eq_symbol_intad)
print(len(differenttad_merged_eq_symbol_intad))

### Plot the ratio of same-tad vs different-tad EQs as a stacked bargraph
ratio_tad_df = pd.DataFrame({
    'type': ['same','different'],
    'number': [188,31]})
display(ratio_tad_df)

ax = sns.histplot(ratio_tad_df,  x='type', weights='number',
              palette='tab20c', shrink=0.8)
ax.set_ylabel('Number')
# Fix the legend so it's not on top of the bars.
legend = ax.get_legend()
legend.set_bbox_to_anchor((1, 1))
outplot = 'Number_TADType_AllSigEQInTADK562_StackedBar.pdf'
plt.savefig(outplot)
plt.close('all')

"""
Notably, the "significant" elements that are not in the same TAD as their genes tend to have much lower significance values
The p-values are close to e-03/04. Their effect sizes are also much smaller, and closer to 0.
Plot this to make a semi-convincing argument that these are false negatives

"""

,chr,start,end,label,EffectSize,pValue,pValueAdjusted,chr2,start2,end2,tad_cc,symbol,gchr,gstart,gend,strand
0,chr19,12886091,12887931,chr19:12996905-12998745:._CALR,0.085682,2.343624e-13,1.335866e-11,chr19,12875000,12955000,chr19_12875000_12955000,CALR,chr19,12937599,12939599,+
1,chr19,12936451,12936951,chr19:13047265-13047765:._CALR,0.084431,7.167944e-04,2.042864e-02,chr19,12875000,12955000,chr19_12875000_12955000,CALR,chr19,12937599,12939599,+
2,chr19,12938211,12938711,chr19:13049025-13049525:._CALR,0.249025,1.401191e-33,1.597357e-31,chr19,12875000,12955000,chr19_12875000_12955000,CALR,chr19,12937599,12939599,+
4,chr19,12881331,12881831,chr19:12992145-12992645:._DNASE2,0.393293,7.151473e-153,8.295708e-151,chr19,12875000,12955000,chr19_12875000_12955000,DNASE2,chr19,12880521,12882521,-
5,chr19,12716871,12717391,chr19:12827685-12828205:._JUNB,-0.068360,5.088608e-04,9.753165e-03,chr19,12695000,12855000,chr19_12695000_12855000,JUNB,chr19,12790495,12792495,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,chr19,48753488,48754588,chr19:49256745-49257845:._FUT1,0.472751,2.551523e-104,1.990188e-102,chr19,48630000,48990000,chr19_48630000_48990000,FUT1,chr19,48752660,48754660,-
215,chr19,48753488,48754588,chr19:49256745-49257845:._FUT1,0.472751,2.551523e-104,1.990188e-102,chr19,48640000,48780000,chr19_48640000_48780000,FUT1,chr19,48752660,48754660,-
216,chr19,48755248,48755768,chr19:49258505-49259025:._FUT1,0.336078,9.140233e-126,1.425876e-123,chr19,48630000,48990000,chr19_48630000_48990000,FUT1,chr19,48752660,48754660,-
217,chr19,48755248,48755768,chr19:49258505-49259025:._FUT1,0.336078,9.140233e-126,1.425876e-123,chr19,48640000,48780000,chr19_48640000_48780000,FUT1,chr19,48752660,48754660,-


,chr,start,end,label,EffectSize,pValue,pValueAdjusted,chr2,start2,end2,tad_cc,symbol,gchr,gstart,gend,strand
3,chr19,12775491,12776011,chr19:12886305-12886825:._DNASE2,0.046131,4.548657e-04,1.758814e-02,chr19,12695000,12855000,chr19_12695000_12855000,DNASE2,chr19,12880521,12882521,-
10,chr19,12886091,12887931,chr19:12996905-12998745:._JUNB,0.037758,1.567537e-05,6.008892e-04,chr19,12875000,12955000,chr19_12875000_12955000,JUNB,chr19,12790495,12792495,+
11,chrX,47711226,47711786,chrX:47570625-47571185:._PIM2,-0.082545,1.393697e-03,2.065206e-02,chrX,47540000,47750000,chrX_47540000_47750000,PIM2,chrX,48918136,48920136,-
24,chr19,12886091,12887931,chr19:12996905-12998745:._WDR83OS,0.067410,2.564174e-14,9.743861e-13,chr19,12875000,12955000,chr19_12875000_12955000,WDR83OS,chr19,12668651,12670651,-
36,chr3,128426282,128426962,chr3:128145125-128145805:._RPN1,0.047574,8.894700e-04,2.632831e-02,chr3,128415000,128490000,chr3_128415000_128490000,RPN1,chr3,128649876,128651876,-
37,chr3,128532862,128533362,chr3:128251705-128252205:._RPN1,-0.092510,4.451220e-04,1.646951e-02,chr3,128500000,128585000,chr3_128500000_128585000,RPN1,chr3,128649876,128651876,-
46,chr19,12886091,12887931,chr19:12996905-12998745:._DHPS,0.041932,2.890382e-16,1.117614e-14,chr19,12875000,12955000,chr19_12875000_12955000,DHPS,chr19,12680887,12682887,-
47,chrX,49192648,49193148,chrX:49049105-49049605:._PQBP1,-0.067643,8.355084e-04,2.255873e-02,chrX,49190000,49270000,chrX_49190000_49270000,PQBP1,chrX,48896911,48898911,+
48,chrX,47542086,47542666,chrX:47401485-47402065:._GATA1,-0.074498,1.058417e-03,2.349686e-02,chrX,47540000,47750000,chrX_47540000_47750000,GATA1,chrX,48785573,48787573,+
49,chrX,49184272,49184772,chrX:49040725-49041225:._GATA1,-0.036268,1.000471e-03,2.349686e-02,chrX,49115000,49185000,chrX_49115000_49185000,GATA1,chrX,48785573,48787573,+


31


,type,number
0,same,188
1,different,31


AttributeError: 'NoneType' object has no attribute 'set_bbox_to_anchor'

In [14]:
### Plot distribution of effect sizes 

# Get effects for each type
effect_sizes_sametad = sametad_merged_eq_symbol_intad[['EffectSize']]
effect_sizes_sametad['type']='same'


effect_sizes_differenttad = differenttad_merged_eq_symbol_intad[['EffectSize']]
effect_sizes_differenttad['type']='different'

# Merge
effect_sizes_merged = pd.concat([effect_sizes_sametad, effect_sizes_differenttad])
display(effect_sizes_merged)

### plot
plt.close('all')

g1 = sns.boxplot(data=effect_sizes_merged, x='type', y='EffectSize')
outplot = 'EffectSizes_SameTADType_AllK562SigEQ.pdf'
#plt.xlim(0, 3000)
plt.savefig(outplot)
plt.close('all')

### Plot distribution of significance
# Get pValueAdjusted significance for each type
sig_sametad = sametad_merged_eq_symbol_intad[['pValueAdjusted']]
sig_sametad['type']='same'

sig_differenttad = differenttad_merged_eq_symbol_intad[['pValueAdjusted']]
sig_differenttad['type']='different'

# Merge
sig_merged = pd.concat([sig_sametad, sig_differenttad])

# -Log10 transform
sig_merged['logtransformsig']= -np.log10(sig_merged['pValueAdjusted'])
display(sig_merged)

### plot
plt.close('all')
g2 = sns.boxplot(data=sig_merged, x='type', y='logtransformsig')
g2.set_ylabel('-Log10 pValueAdjusted')
outplot = 'NegLog10pValueAdjusted_SameTADType_AllK562SigEQ.pdf'
#plt.xlim(0, 3000)
plt.savefig(outplot)
plt.close('all')



<ipython-input-14-fb85a1dde24b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  effect_sizes_sametad['type']='same'
<ipython-input-14-fb85a1dde24b>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  effect_sizes_differenttad['type']='different'


,EffectSize,type
0,0.085682,same
1,0.084431,same
2,0.249025,same
4,0.393293,same
5,-0.068360,same
...,...,...
195,0.031474,different
196,-0.050039,different
197,-0.050039,different
198,-0.046454,different


<ipython-input-14-fb85a1dde24b>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sig_sametad['type']='same'
<ipython-input-14-fb85a1dde24b>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sig_differenttad['type']='different'
/Users/davidy/miniconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


,pValueAdjusted,type,logtransformsig
0,1.335866e-11,same,10.874237
1,2.042864e-02,same,1.689761
2,1.597357e-31,same,30.796598
4,8.295708e-151,same,150.081147
5,9.753165e-03,same,2.010854
...,...,...,...
195,1.193146e-02,different,1.923306
196,2.792178e-02,different,1.554057
197,2.792178e-02,different,1.554057
198,2.326893e-02,different,1.633224


In [15]:
### Run statistics

tstat, pval = stats.ttest_ind(effect_sizes_sametad['EffectSize'],effect_sizes_differenttad['EffectSize'],equal_var=False)
print(tstat, pval)


7.763746357338594 2.8466678561031126e-12
